In [ ]:
%matplotlib widget
from sympy import *
init_printing(use_latex=True)

# 22 - Plotting Module

## 22.3 - Examples

In [ ]:
from spb import *
var("x, y, z")

In [ ]:
plot(sin(x / (cos(10 * x) + 2)), (x, -4, 4))

In [ ]:
plot(
    (sin(x), (x, -10, 10), "s"),
    (cos(x), (x, -10, 10), "c"),
    (log(x), (x, 1e-03, 10), "l"),
    backend=MB
)

In [ ]:
var("u, v")
expr = (
    cos(v) * (6 - (5/4 + sin(3 * u)) * sin(u - 3 * v)),
    (6 - (5/4 + sin(3 * u)) * sin(u - 3 * v)) * sin(v),
    -cos(u - 3 * v) * (5/4 + sin(3 * u))
)
plot3d_parametric_surface(
    *expr, (u, 0, 2 * pi), (v, 0, 2 * pi),
    n=50, backend=MB, use_cm=True
)
# NOTE: if using Matplotlib >= 3.6.0, try adding: aspect="equal"
# To check the version:
# import matplotlib;print(matplotlib.__version__)

In [ ]:
plot3d_parametric_surface(
    *expr, (u, 0, 2 * pi), (v, 0, 2 * pi),
    n=250, backend=PB, use_cm=False,
    wireframe=True, wf_n1=30, wf_n2=30
)

## 22.4 - Modifying and Saving Plots

In [ ]:
var("u, v")
expr = (
    cos(v) * (6 - (5/4 + sin(3 * u)) * sin(u - 3 * v)),
    (6 - (5/4 + sin(3 * u)) * sin(u - 3 * v)) * sin(v),
    -cos(u - 3 * v) * (5/4 + sin(3 * u))
)
p = plot3d_parametric_surface(
    *expr, (u, 0, 2 * pi), (v, 0, 2 * pi),
    n=50, use_cm=False, backend=MB, size=(5, 2.5)
)

In [ ]:
ax = p.ax
ax.collections[0].set_facecolors([0.7] * 3)
ax.collections[0].set_linewidth(0.25)
ax.collections[0].set_edgecolors("k")

In [ ]:
help(p.save)

In [ ]:
p.save("plot3d.pdf")

In [ ]:
p = plot3d_parametric_surface(
    *expr, (u, 0, 2 * pi), (v, 0, 2 * pi),
    n=250, backend=KB
)
p.save("k3d.html", include_js=True)

## 22.5 - Parametric-Interactive Plots

In [ ]:
from spb import *
import param

### 22.5.1 - Example - Fourier Series Aapproximation

In [ ]:
x, T, n, m = symbols("x, T, n, m")
sawtooth = frac(x / T)
# Fourier Series of a sawtooth wave
fs = S(1) / 2 - (1 / pi) * Sum(sin(2 * n * pi * x / T) / n, (n, 1, m))

In [ ]:
# WITH PANEL
from bokeh.models.formatters import PrintfTickFormatter
formatter = PrintfTickFormatter(format="%.3f")
plot(
    (sawtooth, (x, 0, 10), "f"),
    (fs, (x, 0, 10), "approx"),
    params = {
        T: (2, 0, 10, 80, formatter),
        m: param.Integer(3, bounds=(1, None), label="Sum up to n ")
    },
    xlabel = "x",
    ylabel = "y",
    backend = PB,
    imodule = "panel",
    use_latex = False
)

In [ ]:
# WITH IPYWIDGETS
import ipywidgets
plot(
    (sawtooth, (x, 0, 10), "f"),
    (fs, (x, 0, 10), "approx"),
    params = {
        T: (2, 0, 10, 80),
        m: ipywidgets.IntSlider(3, min=1, max=100, description="Sum up to n ")
    },
    xlabel = "x",
    ylabel = "y",
    backend = PB,
)

### 25.5.2 - Example - Temperature Distribution

In [ ]:
r, ro, ri = symbols("r, r_o, r_i")
mdot, cp, hc = symbols(r"\dot{m}, c_p, h_c")
alpha, k, L, z = symbols("alpha, k, L, z")
Tin, Pave = symbols("T_{in}, P_{ave}")
# Fuel temperature distribution along the channel
# here, the only variable is z, everything else are parameters
Tf = (Tin
    + (Pave * L * pi * (ro ** 2 - ri ** 2) / (2 * mdot * cp))
    * (1 - sin(alpha * (L / 2 - z)) / sin(alpha * L / 2))
    + (alpha * Pave * L / 2)
    * (cos(alpha * (L / 2 - z)) / sin(alpha * L / 2))
    * (
    (ro ** 2 - ri ** 2) / (2 * hc * ri)
    - (1 / (2 * k)) * ((r ** 2 - ri ** 2) / 2 + ro ** 2 * log(ri / r))))
# Fuel temperature distribution at the inner and outer walls
Twi = Tf.subs(r, ri)
Two = Tf.subs(r, ro)
# Cooling fluid temperature
Tp = Tin + (Pave * L / 2) * pi * (ro ** 2 - ri ** 2) / (mdot * cp) * (
    1 - sin(alpha * (L / 2 - z)) / sin(alpha * L / 2))
display(Twi, Two, Tp)

In [ ]:
from bokeh.models.formatters import PrintfTickFormatter
f3 = PrintfTickFormatter(format="%.3f")
f5 = PrintfTickFormatter(format="%.5f")
plot(
    (Twi, prange(z, 0, L), "Twi"),
    (Two, prange(z, 0, L), "Two"),
    (Tp, prange(z, 0, L), "Tp"),
    params = {
        ri: (0.2, 0.04, 0.5, 100, f3),
        ro: (0.4, 0.2, 1.6, 100, f3),
        L: (100, 25, 250),
        Pave: (1000, 400, 4000),
        Tin: (300, 100, 500),
        hc: (1, 0.4, 15, 100, f3),
        alpha: (0.031, 0.016, 0.031, 100, f5),
        mdot: (1, 0.5, 5, 100, f3),
        k: (0.2, 0.1, 2, 100, f3),
        cp: (15, 5, 25)
    },
    title = "Temperature distribution",
    xlabel = "Position [cm]",
    ylabel = "T [K]",
    ylim = (0, 3000),
    backend = BB,
    use_latex = False,
    imodule = "panel"
)

In [ ]:
# serves the interactive application on a new browser window
plot(
    (Twi, prange(z, 0, L), "Twi"),
    (Two, prange(z, 0, L), "Two"),
    (Tp, prange(z, 0, L), "Tp"),
    params = {
        ri: (0.2, 0.04, 0.5, 100, f3),
        ro: (0.4, 0.2, 1.6, 100, f3),
        L: (100, 25, 250),
        Pave: (1000, 400, 4000),
        Tin: (300, 100, 500),
        hc: (1, 0.4, 15, 100, f3),
        alpha: (0.031, 0.016, 0.031, 100, f5),
        mdot: (1, 0.5, 5, 100, f3),
        k: (0.2, 0.1, 2, 100, f3),
        cp: (15, 5, 25)
    },
    xlabel = "Position [cm]",
    ylabel = "T [K]",
    ylim = (0, 4500),
    backend = BB,
    use_latex = False,
    imodule = "panel",
    servable = True,
    layout = "sbl", # move the widgets to the left of the window
    ncols = 1, # lay widgets in a single column
    name = "Temperature distribution", # name of the interactive application
    size = (1000, 800) # size of the plot (in pixels for BB)
)